In [183]:
import numpy as np
import pandas as pd
import math
import os
import re

In [142]:
os.chdir('/Users/xll/Desktop/data')
os.getcwd()

'/Users/xll/Desktop/data'

# 一、数据预处理
## 1、年个股回报率文件

In [143]:
dt1 = pd.read_excel('TRD_Year.xlsx',"Sheet1",skiprows = [1,2])

In [144]:
stk_E = dt1[dt1['Stkcd']//10000 == 60].rename(columns = {'Ysmvttl':'E'})

In [254]:
stk_E = stk_E[~np.isnan(stk_E['E'])]
stk_E['E'] = stk_E['E'] * 1000

In [146]:
stkls = stk_E['Stkcd']

## 2、月个股回报率文件

In [210]:
dt2 = pd.read_excel('TRD_Mnth.xlsx',"Sheet1",skiprows = [1,2])

In [211]:
stk_P = dt2[dt2['Stkcd'].isin(stkls)]

In [212]:
stk_std = stk_P.groupby('Stkcd').std().reset_index().rename(columns={'Mclsprc':'std'})

In [213]:
stk_mean = stk_P.groupby('Stkcd').mean().reset_index().rename(columns={'Mclsprc':'mean'})

In [218]:
stk_P = pd.merge(stk_std, stk_mean, how = 'inner', on = 'Stkcd')
stk_P['PriceTheta'] = stk_P['std']/stk_P['mean'] * math.sqrt(12)

In [268]:
stk_P = stk_P[~np.isnan(stk_P['PriceTheta'])][['Stkcd','PriceTheta']]

## 3、资产负债表

In [151]:
dt3 = pd.read_excel('FS_Combas.xlsx',"Sheet1",skiprows = [1,2])

In [152]:
stk_D = dt3[(dt3['Stkcd'].isin(stkls))&(dt3['Typrep'] == 'A')].rename(columns = {'A001100000':'SD','A001200000':'LD'})

In [153]:
stk_D['D'] = stk_D['SD'] + 0.5 * stk_D['LD'] 

In [154]:
stk_D = stk_D[~np.isnan(stk_D['D'])][['Stkcd','D']]

## 4、金融机构存款利率表

In [155]:
dt4 = pd.read_excel('CMMPI_Irfid.xlsx',"Sheet1",skiprows = [1,2])

In [263]:
rf = 1.1 / 100

## 5、合并

In [375]:
stks = pd.merge(stk_E, stk_P, how = 'inner', on = ['Stkcd'])

In [376]:
stks = pd.merge(stks, stk_D, how = 'inner', on = ['Stkcd'])

In [377]:
stk_sp = stks.sample(n=200)

In [1]:
des= stk_sp.ix[:,1:6].describe()

NameError: name 'stk_sp' is not defined

In [379]:
des.to_csv('describe.csv')

In [380]:
stk_sp['r'] = rf

In [381]:
stk_sp['T'] = 1

In [382]:
stk_sp.to_csv('data.csv')

# 二、计算结果描述

In [391]:
result= pd.read_excel('result.xlsx')

In [394]:
rs_ds= result.ix[:,1:3].describe().applymap(lambda x: round(x,2))

In [395]:
rs_ds.to_csv('result.csv')

# 三、解释变量数据采集

In [500]:
sp = pd.read_csv('data.csv')

In [501]:
exp = sp[['Stkcd','E']]

In [502]:
dt5 = pd.read_excel('FS_Comins.xlsx',"Sheet1",skiprows = [1,2])
exp = pd.merge(exp, dt5[['Stkcd','B002000000']], how = 'inner', on = 'Stkcd').rename(columns = {'B002000000':'ret'})

In [503]:
dt6 = pd.read_excel('FS_Combas2.xlsx',"Sheet1",skiprows = [1,2]).rename(columns = {'A001123000': 'str','A001100000':'SA','A001000000':'TA','A002100000':'SD','A002000000':'TD'})

In [504]:
dt7 = dt6[dt6['Stkcd'].isin(stk_sp['Stkcd'])]
dt7['Acid'] = (dt7['SA'] - dt7['str'])/dt7['SD']
dt7['TD/TA'] = dt7['TD']/dt7['TA']

/Users/xll/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/xll/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [505]:
exp = pd.merge(exp, dt7[['Stkcd','Acid','TD/TA','TD']], how = 'left', on = 'Stkcd')
exp = pd.merge(exp, result[['Stkcd','EDF']], how = 'left', on = 'Stkcd')

In [506]:
exp.to_csv('expdata.csv')

In [509]:
expdes = exp.ix[:,1:7].describe()

In [510]:
expdes.to_csv('expdes.csv')